In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import logging
import os
from datetime import datetime
from pathlib import Path

import json

from dotenv import load_dotenv

from econagents.core.game_runner import GameRunner, HybridGameRunnerConfig
from examples.ibex_tudelft.futarchy.manager import HLAgentManager
from examples.ibex_tudelft.create_game import create_game_from_specs

load_dotenv()

# Load environment variables
HOSTNAME = os.getenv("HOSTNAME")
PORT = os.getenv("PORT")
USERNAME = os.getenv("GAME_USERNAME")
PASSWORD = os.getenv("GAME_PASSWORD")

In [ ]:
for i in range(1, 8):
    print(f"begin round, {i}")
    # set reference to correct dataset
    with open(Path().absolute() / "futarchy.json", "r") as jsonFile:
        data = json.load(jsonFile)
        data["session_number"] = i

    with open(Path().absolute() / "futarchy.json", "w") as jsonFile:
        json.dump(data, jsonFile)

    # Create a new game
    new_game_data = create_game_from_specs(
        specs_path=Path().absolute() / "futarchy.json",
        base_url=f"http://{HOSTNAME}",
        game_name=f"futarchy_sim_{i} {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        credentials={"username": USERNAME, "password": PASSWORD},
    )

    game_id = new_game_data["game_id"]
    num_agents = new_game_data["num_agents"]
    login_payloads = new_game_data["login_payloads"]

    # Run the game
    config = HybridGameRunnerConfig(
        # Game configuration
        game_id=game_id,
        # Server configuration
        protocol="ws",
        hostname=HOSTNAME,
        port=int(PORT),
        path="wss",
        # Continuous-time phase configuration
        continuous_phases=[2, 5],
        min_action_delay=2,
        max_action_delay=7,
        # Logging configuration
        log_level=logging.DEBUG,
    )
    agents = [HLAgentManager(game_id=game_id, auth_mechanism_kwargs=login_payloads[i]) for i in range(num_agents)]
    runner = GameRunner(agents=agents, config=config)
    await runner.run_game()
    print(f"end round, {i}")